In [ ]:
# Librerie Generiche

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from time import time
import seaborn as sn

# Keras
import keras
from keras.utils import to_categorical

# Sklearn
import sklearn
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV,  RepeatedStratifiedKFold
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, accuracy_score, roc_curve, classification_report
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.utils.estimator_checks import check_estimator

np.random.seed(1234)
tf.random.set_seed(1234)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
Music = pd.read_csv('/content/Music_3_seconds_datatset.csv')
Music = Music.drop(columns=['Unnamed: 0'])

In [ ]:
Music

,filename,Mean_ZCR,Var_ZCR,Mean_Harmonic,Var_Harmonic,Mean_Percussive,Var_Percussive,BMP,Mean_Chroma,Var_Chroma,Mean_RMSE,Var_RMSE,Mean_Centroid,Var_Centroid,Mean_Bandwidth,Var_Bandwidth,Mean_Rolloff,Var_Rolloff,Mean_MFCC_1,Mean_MFCC_2,Mean_MFCC_3,Mean_MFCC_4,Mean_MFCC_5,Mean_MFCC_6,Mean_MFCC_7,Mean_MFCC_8,Mean_MFCC_9,Mean_MFCC_10,Mean_MFCC_11,Mean_MFCC_12,Mean_MFCC_13,Mean_MFCC_14,Mean_MFCC_15,Mean_MFCC_16,Mean_MFCC_17,Mean_MFCC_18,Mean_MFCC_19,Mean_MFCC_20,Var_MFCC_1,Var_MFCC_2,Var_MFCC_3,Var_MFCC_4,Var_MFCC_5,Var_MFCC_6,Var_MFCC_7,Var_MFCC_8,Var_MFCC_9,Var_MFCC_10,Var_MFCC_11,Var_MFCC_12,Var_MFCC_13,Var_MFCC_14,Var_MFCC_15,Var_MFCC_16,Var_MFCC_17,Var_MFCC_18,Var_MFCC_19,Var_MFCC_20,Genre
0,blues.00000.0.wav,0.081851,0.000558,-0.000079,0.008377,-0.000067,0.005527,129.199219,0.371545,0.097370,0.130405,0.003521,1773.285877,168244.728448,1970.642720,117376.557123,3689.714355,1.084507e+06,-118.630563,125.074785,-23.448579,41.323168,-5.962212,20.120882,-16.048139,17.865841,-14.624666,10.264737,-9.710334,10.172429,-4.694758,8.414781,-7.225894,-2.849642,-3.241224,0.721985,-5.043307,-0.237658,2440.249767,262.009119,363.937488,181.639034,152.701542,75.747361,40.196536,84.356901,83.432848,96.787234,66.676815,45.036079,34.328806,48.219525,42.720911,39.725565,36.486435,38.096757,33.608330,43.827777,Blues
1,blues.00000.1.wav,0.087173,0.001030,-0.000099,0.004950,-0.000103,0.004854,123.046875,0.358604,0.088067,0.112699,0.001450,1816.195860,90703.325185,2007.028054,65263.468497,3838.541917,6.702624e+05,-125.693258,122.533665,-20.813553,50.239900,-11.454327,21.517383,-16.060025,19.526212,-12.590309,18.012476,-11.787571,8.219579,-7.826589,8.475853,-8.353868,4.123801,-6.100085,0.219240,-2.875457,5.800726,2035.793354,216.652804,228.939817,144.013383,137.454883,80.159669,98.829273,57.284719,64.623654,52.109480,54.595104,41.329198,29.287178,48.245444,68.184197,65.312978,40.738823,50.587802,96.975451,60.006120,Blues
2,blues.00000.2.wav,0.071383,0.000425,-0.000066,0.012495,0.000066,0.004332,123.046875,0.369653,0.088131,0.132002,0.004620,1788.642783,111322.537051,2080.274699,74844.653404,3955.152494,7.869501e+05,-132.379473,115.051511,-14.793070,50.130156,-0.622686,24.620805,-13.497784,15.592506,-13.192537,10.359210,-7.947918,11.867365,-4.614052,6.582664,-2.380736,4.789980,-1.801320,2.308314,-1.931481,2.522431,3803.214390,254.885063,192.533512,148.520290,124.937107,67.538015,89.566209,55.093591,119.513865,36.192966,47.171665,36.335747,42.669878,31.066391,50.024958,67.582202,28.132218,48.102488,53.116809,33.136236,Blues
3,blues.00000.3.wav,0.069426,0.000304,-0.000015,0.008305,0.000019,0.005937,123.046875,0.375004,0.085761,0.132562,0.002447,1654.902168,112316.264385,1955.041792,83615.398343,3530.617112,9.115652e+05,-118.270070,132.186076,-18.827093,39.810047,-13.315146,20.517824,-14.611798,18.795087,-13.820897,13.044023,-12.688901,10.437570,-4.401806,9.172530,-9.931134,-1.322859,-3.859235,1.255384,-3.619867,3.639336,2507.519772,335.775290,109.647862,184.911773,144.002532,120.346076,70.858321,72.604434,73.689240,42.435005,57.015278,39.020790,60.470406,40.195781,44.049540,47.059677,27.984930,35.093279,51.420635,31.930043,Blues
4,blues.00000.4.wav,0.070102,0.000288,0.000041,0.009632,-0.000106,0.005834,123.046875,0.339656,0.087812,0.143289,0.001701,1630.737017,79648.228297,1944.895113,60292.824395,3439.432279,6.096879e+05,-105.953849,134.640774,-19.983114,40.192189,-14.282934,18.724304,-15.583555,19.210046,-18.246399,14.255729,-5.821648,17.055470,-5.683838,5.760249,-7.993854,2.090513,0.629567,1.709780,-3.402106,0.546949,2118.730026,219.527474,172.722627,102.951567,102.458607,79.122140,49.016389,53.547850,95.413792,60.658896,53.711265,43.340779,46.452456,24.728624,39.796112,30.307622,44.883642,51.706692,26.686307,29.212284,Blues
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [ ]:
dati_X = Music.drop(columns=['Genre', 'filename'])
X = dati_X.values
min_max_scaler_x = sklearn.preprocessing.MinMaxScaler()
X_scale = min_max_scaler_x.fit_transform(X)
X_scale

array([[0.20445758, 0.00847825, 0.62778287, ..., 0.05530271, 0.02661089,
        0.04782937],
       [0.22038394, 0.01571868, 0.62731684, ..., 0.07519716, 0.08208627,
        0.06559999],
       [0.17313312, 0.00644692, 0.62809121, ..., 0.0712388 , 0.04368978,
        0.03608556],
       ...,
       [0.17762892, 0.01006393, 0.62884734, ..., 0.04410298, 0.03992184,
        0.04251938],
       [0.22618352, 0.01506607, 0.62902882, ..., 0.08108372, 0.05268414,
        0.02029411],
       [0.17556085, 0.05798272, 0.62999914, ..., 0.03597615, 0.03947958,
        0.04554711]])

In [ ]:
Y = Music['Genre']
LE = sklearn.preprocessing.LabelEncoder()
Y_label = LE.fit_transform(Y)

In [ ]:
X_scale.shape

(9980, 57)

In [ ]:
Y_label.shape

(9980,)

In [ ]:
X_train, X_test , y_train, y_test = train_test_split(X_scale, Y_label, test_size=0.2)

In [ ]:
X_train.shape

(7984, 57)

In [ ]:
X_test.shape

(1996, 57)

In [ ]:
y_train.shape

(7984,)

In [ ]:
y_test.shape

(1996,)

In [ ]:
y_train[0]

1

In [ ]:
# SVM

# Random Search

# Parameters to be tested in cross-validation
param_grid = {'C': [1e2, 5e2, 1e3],
          'gamma': [0.005, 0.01, 0.1, 0.5, 1.0], }

# Support Vector Machine initialization
clf_SVC = GridSearchCV(SVC(kernel='rbf', class_weight='balanced'), param_grid, cv=2)

# Training
t0 = time()
clf_SVC = clf_SVC.fit(X_train, y_train)
print('Training completed in %0.3fs' % (time() - t0))
# Test
y_pred_SVC = clf_SVC.predict(X_test)

In [ ]:
target_names = ['Blues', 'Classical', 'Country', 'Disco', 'Hiphop', 'Jazz', 'Metal', 'Pop', 'Reggae', 'Rock']
# Classificaiton report
print('Classification report:')
print(classification_report(y_test, y_pred_SVC, target_names= target_names))

# Confusion matrix
print('Confusion matrix:')
cm = confusion_matrix(y_test, y_pred_SVC)
print(cm)

Classification report:
              precision    recall  f1-score   support

       Blues       0.90      0.96      0.93       208
   Classical       0.96      0.99      0.98       218
     Country       0.88      0.85      0.87       192
       Disco       0.89      0.94      0.91       189
      Hiphop       0.89      0.92      0.90       193
        Jazz       0.93      0.96      0.94       205
       Metal       0.95      0.99      0.97       203
         Pop       0.94      0.87      0.90       200
      Reggae       0.93      0.90      0.92       187
        Rock       0.89      0.78      0.83       201

    accuracy                           0.92      1996
   macro avg       0.92      0.92      0.91      1996
weighted avg       0.92      0.92      0.92      1996

Confusion matrix:
[[199   0   2   2   2   0   0   0   2   1]
 [  0 216   0   0   0   2   0   0   0   0]
 [  4   3 164   0   1   6   0   2   3   9]
 [  1   2   2 177   0   0   1   1   2   3]
 [  0   0   1   4 177   0   

In [ ]:
# Result of the cross validation for parameters selection SVM
print('Best parameters combination SVM:')
print(' C: '+str(clf_SVC.best_estimator_.C))
print(' gamma: '+str(clf_SVC.best_estimator_.gamma))

Best parameters combination SVM:
 C: 100.0
 gamma: 1.0


In [ ]:
# LOGISTIC REGRESSION

# Random Search

space = dict()
space['solver'] = ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
space['penalty'] = ['none', 'l1', 'l2', 'elasticnet']
space['C'] = (range(1,1000))
space['multi_class'] = ['auto', 'ovr', 'multinomial']
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
model = LogisticRegression()

clf_LR = RandomizedSearchCV(model, space, scoring='accuracy', n_jobs=-1)
# Training
clf_LR = clf_LR.fit(X_train, y_train)

# Pred
y_pred_LR = clf_LR.predict(X_test)

In [ ]:
# Classificaiton report
print('Classification report:')
print(classification_report(y_test, y_pred_LR, target_names=target_names))

# Confusion matrix
print('Confusion matrix:')
cm_LR = confusion_matrix(y_test, y_pred_LR)
print(cm_LR)

Classification report:
              precision    recall  f1-score   support

       Blues       0.67      0.63      0.65       208
   Classical       0.94      0.95      0.95       218
     Country       0.62      0.65      0.63       192
       Disco       0.63      0.69      0.66       189
      Hiphop       0.70      0.63      0.66       193
        Jazz       0.78      0.82      0.80       205
       Metal       0.77      0.89      0.82       203
         Pop       0.78      0.81      0.80       200
      Reggae       0.60      0.60      0.60       187
        Rock       0.56      0.41      0.47       201

    accuracy                           0.71      1996
   macro avg       0.70      0.71      0.70      1996
weighted avg       0.71      0.71      0.71      1996

Confusion matrix:
[[132   1  11   7   6   9  23   0   9  10]
 [  1 207   3   1   1   3   0   0   0   2]
 [ 15   1 124   9   2  11   2   3  11  14]
 [  1   2   5 131   7   0   1  12   8  22]
 [  5   0   5  10 122   1   

In [ ]:
# Result of the cross validation for parameters selection Logistic Regression
print('Best parameters combination Logistic Regression:')
print(' C: '+str(clf_LR.best_estimator_.C))
print(' Penalty: '+str(clf_LR.best_estimator_.penalty))
print(' Solver: '+str(clf_LR.best_estimator_.solver))
print(' Solver: '+str(clf_LR.best_estimator_.multi_class))

Best parameters combination Logistic Regression:
 C: 520
 Penalty: l2
 Solver: newton-cg
 Solver: ovr


In [ ]:
# Random Forrest

# Random Search

space = dict()
space['n_estimators'] = (range(0,1000))
space['criterion']= ['gini', 'entropy']
space['min_samples_split'] = (range(2,10))
space['min_samples_leaf'] = (range(1,10))
space['max_features'] = ["auto", "sqrt", "log2"]
space['bootstrap'] = ['True', 'False']
space['oob_score'] = ['True', 'False']
space['warm_start'] = ['True', 'False']

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
model = RandomForestClassifier()

clf_RF = RandomizedSearchCV(model, space, scoring='accuracy', n_jobs=-1)
# Training
clf_RF = clf_RF.fit(X_train, y_train)

# Pred
y_pred_RF = clf_RF.predict(X_test)

In [ ]:
# Classificaiton report
print('Classification report:')
print(classification_report(y_test, y_pred_RF, target_names=target_names))

# Confusion matrix
print('Confusion matrix:')
cm_RF = confusion_matrix(y_test, y_pred_RF)
print(cm_RF)

Classification report:
              precision    recall  f1-score   support

       Blues       0.90      0.84      0.87       208
   Classical       0.93      0.97      0.95       218
     Country       0.80      0.82      0.81       192
       Disco       0.87      0.88      0.87       189
      Hiphop       0.92      0.87      0.89       193
        Jazz       0.85      0.90      0.87       205
       Metal       0.90      0.96      0.93       203
         Pop       0.90      0.90      0.90       200
      Reggae       0.82      0.88      0.85       187
        Rock       0.92      0.77      0.83       201

    accuracy                           0.88      1996
   macro avg       0.88      0.88      0.88      1996
weighted avg       0.88      0.88      0.88      1996

Confusion matrix:
[[175   0   8   4   2   5   8   0   4   2]
 [  0 211   1   0   0   4   0   0   1   1]
 [  6   5 158   2   1   9   1   0   9   1]
 [  0   1   5 167   6   2   0   4   3   1]
 [  2   0   3   4 168   0   

In [ ]:
# Result of the cross validation for parameters selection Random Forrest
print('Best parameters combination Random Forrest:')
print(' n_estimators: '+str(clf_RF.best_estimator_.n_estimators))
print(' criterion: '+str(clf_RF.best_estimator_.criterion))
print(' min_samples_split: '+str(clf_RF.best_estimator_.min_samples_split))
print(' min_samples_leaf: '+str(clf_RF.best_estimator_.min_samples_leaf))
print(' max_features: '+str(clf_RF.best_estimator_.max_features))
print(' bootstrap: '+str(clf_RF.best_estimator_.bootstrap))
print(' oob_score: '+str(clf_RF.best_estimator_.oob_score))
print(' warm_start: '+str(clf_RF.best_estimator_.warm_start))

Best parameters combination Random Forrest:
 n_estimators: 428
 criterion: gini
 min_samples_split: 3
 min_samples_leaf: 1
 max_features: auto
 bootstrap: False
 oob_score: True
 warm_start: True


In [ ]:
# Decision Tree

# Random Search

space = dict()
space['splitter'] = ['best', 'random']
space['criterion']= ['gini', 'entropy']
space['min_samples_split'] = (range(2,100))
space['min_samples_leaf'] = (range(1,100))
space['max_features'] = ["auto", "sqrt", "log2"]

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
model = DecisionTreeClassifier()

clf_DT = RandomizedSearchCV(model, space, scoring='accuracy', n_jobs=-1)
# Training
clf_DT = clf_DT.fit(X_train, y_train)

# Pred
y_pred_DT = clf_DT.predict(X_test)

In [ ]:
# Classificaiton report
print('Classification report:')
print(classification_report(y_test, y_pred_DT, target_names=target_names))

# Confusion matrix
print('Confusion matrix:')
cm_DT = confusion_matrix(y_test, y_pred_DT)
print(cm_DT)

Classification report:
              precision    recall  f1-score   support

       Blues       0.38      0.38      0.38       208
   Classical       0.81      0.83      0.82       218
     Country       0.33      0.37      0.35       192
       Disco       0.55      0.44      0.49       189
      Hiphop       0.49      0.53      0.51       193
        Jazz       0.52      0.56      0.54       205
       Metal       0.67      0.68      0.68       203
         Pop       0.70      0.67      0.68       200
      Reggae       0.42      0.36      0.39       187
        Rock       0.40      0.41      0.41       201

    accuracy                           0.53      1996
   macro avg       0.53      0.52      0.52      1996
weighted avg       0.53      0.53      0.53      1996

Confusion matrix:
[[ 79   8  37   7  15  15  22   1  12  12]
 [  0 181   7   1   3  22   1   0   0   3]
 [ 33   5  71  17   0  24   7   1  13  21]
 [  6   4  19  83  26   7   3   8  16  17]
 [ 13   1   8   8 103   1   

In [ ]:
# Result of the cross validation for parameters selection Decision Tree
print(' splitter: '+str(clf_DT.best_estimator_.splitter))
print(' criterion: '+str(clf_DT.best_estimator_.criterion))
print(' min_samples_split: '+str(clf_DT.best_estimator_.min_samples_split))
print(' min_samples_leaf: '+str(clf_DT.best_estimator_.min_samples_leaf))
print(' max_features: '+str(clf_DT.best_estimator_.max_features))

Best parameters combination Random Forrest:
 splitter: best
 criterion: gini
 min_samples_split: 96
 min_samples_leaf: 2
 max_features: auto


In [ ]:
# KNeighbors Classifier

# Random Search

space = dict()
space['n_neighbors'] = range(5,50)
space['algorithm']= ['auto', 'ball_tree', 'kd_tree', 'brute']
space['p'] = [1, 2]

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
model = KNeighborsClassifier()

clf_KN = RandomizedSearchCV(model, space, scoring='accuracy', n_jobs=-1)
# Training
clf_KN = clf_KN.fit(X_train, y_train)

# Pred
y_pred_KN = clf_KN.predict(X_test)

In [ ]:
# Classificaiton report
print('Classification report:')
print(classification_report(y_test, y_pred_KN, target_names=target_names))

# Confusion matrix
print('Confusion matrix:')
cm_KN = confusion_matrix(y_test, y_pred_KN)
print(cm_KN)

Classification report:
              precision    recall  f1-score   support

       Blues       0.90      0.87      0.88       208
   Classical       0.89      0.97      0.93       218
     Country       0.81      0.86      0.83       192
       Disco       0.81      0.92      0.86       189
      Hiphop       0.89      0.90      0.89       193
        Jazz       0.89      0.86      0.88       205
       Metal       0.96      0.94      0.95       203
         Pop       0.93      0.85      0.89       200
      Reggae       0.87      0.89      0.88       187
        Rock       0.90      0.80      0.85       201

    accuracy                           0.89      1996
   macro avg       0.89      0.89      0.89      1996
weighted avg       0.89      0.89      0.89      1996

Confusion matrix:
[[180   1  11   4   0   1   0   0   8   3]
 [  0 211   0   0   0   6   0   0   0   1]
 [  5   6 165   5   1   5   0   0   4   1]
 [  0   2   1 173   5   1   1   1   3   2]
 [  1   0   3   4 174   1   

In [ ]:
# Result of the cross validation for parameters selection KNeighbors Classifier
print('Best parameters combination KNeighbors Classifier:')
print(' n_neighbors: '+str(clf_KN.best_estimator_.n_neighbors))
print(' algorithm: '+str(clf_KN.best_estimator_.algorithm))
print(' mp: '+str(clf_KN.best_estimator_.p))

Best parameters combination KNeighbors Classifier:
 n_neighbors: 7
 algorithm: auto
 mp: 2


In [ ]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation

model = Sequential()
model.add(Dense(1024, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(10, activation='softmax'))
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, epochs=100, batch_size=128)

Epoch 1/100
63/63 [==============================] - 2s 2ms/step - loss: 1.9334 - accuracy: 0.3050
Epoch 2/100
63/63 [==============================] - 0s 2ms/step - loss: 1.2557 - accuracy: 0.5288
Epoch 3/100
63/63 [==============================] - 0s 2ms/step - loss: 1.0993 - accuracy: 0.5975
Epoch 4/100
63/63 [==============================] - 0s 2ms/step - loss: 0.9858 - accuracy: 0.6408
Epoch 5/100
63/63 [==============================] - 0s 2ms/step - loss: 0.8979 - accuracy: 0.6812
Epoch 6/100
63/63 [==============================] - 0s 2ms/step - loss: 0.9258 - accuracy: 0.6663
Epoch 7/100
63/63 [==============================] - 0s 2ms/step - loss: 0.8277 - accuracy: 0.7047
Epoch 8/100
63/63 [==============================] - 0s 2ms/step - loss: 0.8083 - accuracy: 0.7151
Epoch 9/100
63/63 [==============================] - 0s 2ms/step - loss: 0.7849 - accuracy: 0.7194
Epoch 10/100
63/63 [==============================] - 0s 2ms/step - loss: 0.7650 - accuracy: 0.7294
Epoch 11/

In [ ]:
test_loss, test_acc  = model.evaluate(X_test, y_test, batch_size=128)

16/16 [==============================] - 0s 2ms/step - loss: 0.5205 - accuracy: 0.8788
